This document provides a running example of completing the Week 3 assignment....

In [1]:
from pyspark.sql import SQLContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

Import Data

In [2]:
sqlContext = SQLContext(sc)
data = sqlContext.read.load('file:///home/cloudera/Downloads/w3_clustering.csv', 
                          format='com.databricks.spark.csv', 
                          header='true',inferSchema='true')

Let us display the number of lines in the data:

In [3]:
data.count()

709

What does the data contain?

In [4]:
data

DataFrame[totalAdClicks: int, revenue: double]

Statistics about the data:

In [5]:
data.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
totalAdClicks,709,32.208744710860366,16.384120817042554,1,73
revenue,709,44.64880112834979,44.944528765285284,1.0,278.0


Drop all rows with NULL or NaN values...

In [6]:
data = data.na.drop()

Use VectorAssembler to gather all the features:

In [7]:
featuresUsed = ['totalAdClicks', 'revenue']
assembler = VectorAssembler(inputCols=featuresUsed, outputCol="features_unscaled")
assembled = assembler.transform(data)

Scale the features using StandardScaler:

In [8]:
scaler = StandardScaler(inputCol="features_unscaled", outputCol="features", withStd=True, withMean=True)
scalerModel = scaler.fit(assembled)
scaledData = scalerModel.transform(assembled)

Select the features column make the data persist:

In [9]:
scaledData = scaledData.select("features")
scaledData.persist()

DataFrame[features: vector]

We can now perform KMeans clustering to generate 2 clusters:

In [10]:
kmeans = KMeans(k=2, seed=1)
model = kmeans.fit(scaledData)
transformed = model.transform(scaledData)

Print the center of these two clusters:

In [11]:
centers = model.clusterCenters()
centers

[array([ 0.84174521,  0.51884657]), array([-0.79780796, -0.49176392])]

Each array denotes the center for a cluster:

One Cluster is centered at ... array([ 0.84174521, 0.51884657])
Other Cluster is centered at ... array([-0.79780796, -0.49176392])

The first number (field1) in each array refers to scaled verson of the number of ad-clicks and the second number (field2) is the scaled version of the revenue per user. Compare the 1st number of each cluster to see how differently users in each cluster behave when it comes to clicking ads. Compare the 2nd number of each cluster to see how differently users in each cluster behave when it comes to buying stuff.

In one cluster, in general, players click on ads much more often and spend more money on in-app purchases. Assuming that Eglence Inc. gets paid for showing ads and for hosting in-app purchase items, we can use this information to increase game's revenue by increasing the prices for ads we show to the frequent-clickers, and charge higher fees for hosting the in-app purchase items shown to the higher revenue generating buyers.

Note: This analysis requires you to compare the cluster centers and find any ‘significant’ differences in the corresponding feature values of the centers. The answer to this question will depend on the features you have chosen. 

Some features help distinguish the clusters remarkably while others may not tell you much. At this point, if you don’t find clear distinguishing patterns, perhaps re-running the clustering model with different numbers of clusters and revising the features you picked would be a good idea.
